# Sprint 1: Camera Calibration Module

**Project:** Road Defect Detection System (PROSIT 1)  
**Team Members:**
- Naa Lamle Boye
- Thomas Kojo Quarshie
- Chelsea Owusu
- Elijah Boateng

**Date:** 2024

## Purpose

This notebook performs camera calibration to determine the intrinsic parameters (camera matrix K) and distortion coefficients of the smartphone camera used for road defect detection.

**What this notebook achieves:**
- Extracts checkerboard corner points from calibration videos
- Calculates camera intrinsic matrix (focal length, principal point)
- Calculates distortion coefficients (radial and tangential distortion)
- Saves calibration data for use in subsequent processing steps
- Tests calibration by undistorting sample images

**Output:** `camera_calib.npz` file containing calibration parameters (mtx, dist, rvecs, tvecs)

## Step 1: Install Required Packages

In [1]:
!pip install opencv-python numpy

In [2]:
!pip install matplotlib

## Step 2: Setup and Parameters

Define checkerboard dimensions and calibration settings. The checkerboard pattern is used because it provides easily detectable corner points with known 3D coordinates.

In [3]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt

In [4]:
# Configuration parameters
# Number of INTERNAL corners (width, height)
# For a 9x6 square board, there are 8x5 internal corners.
CHECKERBOARD = (8, 5) 

# Termination criteria for sub-pixel accuracy
# We stop when either the accuracy reaches 0.001 or 30 iterations are met.
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(7,4,0)
# These represent the coordinates of the corners in the real world.
# Z=0 because the checkerboard is flat (planar)
objp = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)

# Arrays to store vectors from all calibration images
objpoints = []  # 3D point in real world space
imgpoints = []  # 2D points in image plane

## Step 3: Define Calibration Video Files

Specify the paths to calibration videos. These videos contain the checkerboard pattern captured from different angles and distances to ensure robust calibration.

In [5]:
video_files = [
    'checkerboard/IMG_1059.MOV', 
    'checkerboard/IMG_1060.MOV', 
    'checkerboard/IMG_1061.MOV', 
    'checkerboard/IMG_1062.MOV'
]

## Step 4: Extract Corner Points from Videos

This is the main data collection step. We process frames from each calibration video, detect checkerboard corners, and refine corner positions to sub-pixel accuracy. We sample every 3rd frame to balance between data diversity and computational efficiency.

In [6]:
found_count = 0
sample_rate = 3  # Process every 3rd frame to balance diversity and speed
image_size = None

for video_path in video_files:
    print(f"Processing: {video_path}")
    cap = cv2.VideoCapture(video_path)
    
    frame_in_video = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_in_video % sample_rate == 0:
            # Convert to grayscale (corner detection works on grayscale)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # Store image size (needed for calibration, same for all frames)
            if image_size is None:
                image_size = gray.shape[::-1]  # (width, height)

            # Find the checkerboard corners
            # ADAPTIVE_THRESH handles varying lighting conditions
            # NORMALIZE_IMAGE normalizes the image before detection
            # FAST_CHECK speeds up processing for frames where board isn't visible
            ret_corners, corners = cv2.findChessboardCorners(
                gray, CHECKERBOARD, 
                cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE + cv2.CALIB_CB_FAST_CHECK
            )

            if ret_corners:
                # Refine corner positions to sub-pixel accuracy
                # This improves calibration quality
                corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
                
                # Store the point correspondences
                objpoints.append(objp)  # Same 3D points for all images
                imgpoints.append(corners2)  # Refined 2D points
                found_count += 1
        
        frame_in_video += 1
    cap.release()

print(f"\nTotal valid frames collected across all videos: {found_count}")

Processing: checkerboard/IMG_1059.MOV
Processing: checkerboard/IMG_1060.MOV
Processing: checkerboard/IMG_1061.MOV
Processing: checkerboard/IMG_1062.MOV

Total valid frames collected across all videos: 30


## Step 5: Perform Camera Calibration

Using the collected point correspondences, we calculate the camera intrinsic matrix and distortion coefficients. The RMS reprojection error indicates calibration quality (lower is better, ideally < 1.0 pixel).

In [ ]:
# Perform the camera calibration
# ret: the RMS re-projection error (lower is better, ideally < 1.0)
# mtx: Camera Matrix (contains focal length and principal point)
# dist: Distortion Coefficients (models lens distortion)
# rvecs / tvecs: Rotation and Translation vectors (extrinsic parameters for each image)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, image_size, None, None)

print("--- Calibration Results ---")
print(f"\nRe-projection Error: {ret:.4f}")
print("\nCamera Matrix (K):")
print(mtx)
print("\nDistortion Coefficients (d):")
print(dist)

# Save these results so you don't have to re-run the video processing again
np.savez("camera_calib.npz", mtx=mtx, dist=dist, rvecs=rvecs, tvecs=tvecs)
print("\nCalibration data saved to 'camera_calib.npz'")

## Step 6: Test Calibration by Undistorting an Image

We verify our calibration works by undistorting a sample frame. This removes lens distortion and should make straight lines appear straight.

In [ ]:
# Grab a sample frame from the first calibration video
cap = cv2.VideoCapture(video_files[0])
ret, frame = cap.read()
cap.release()

if ret:
    h, w = frame.shape[:2]

    # Refine the camera matrix 
    # alpha=0: zooms in to remove all black pixels (crops)
    # alpha=1: keeps all pixels but leaves black edges where data is missing
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

    # Apply undistortion to remove lens distortion
    dst = cv2.undistort(frame, mtx, dist, None, newcameramtx)

    # Crop the image based on the ROI (Region of Interest) to remove black edges
    x, y, w_roi, h_roi = roi
    dst_cropped = dst[y:y+h_roi, x:x+w_roi]

    # Display the comparison
    plt.figure(figsize=(15, 10))
    plt.subplot(121), plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)), plt.title('Original (Distorted)')
    plt.subplot(122), plt.imshow(cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)), plt.title('Undistorted')
    plt.show()
else:
    print("Could not read frame.")